# Exploratory Data Analysis

# Setup

In [ ]:
import pandas as pd
import janitor
import os
import sys
from dotenv import load_dotenv
import time
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from eda_utils import add_gbp_columns, explode_lists, get_longest_values, print_in_rows
from stats_builder import make_summary_df, calculate_stats, make_calculated_df, format_stats, format_df
from plots_builder import make_bar_chart
from utils import get_table_from_supabase

#get keys from env
load_dotenv()
url = os.getenv("SUPABASE_URL")
key = os.getenv("SUPABASE_KEY")

# Retrieving Data from Supabase and Building Dataframes

I will connect to Supabase and retrieve all records, in order to start building my analysis dataframes. I will create one dataframe for funder information, and another for grants and recipients information.

In [ ]:
#get tables and build dataframes
tables = ["funders", "causes", "areas", "beneficiaries", "grants",
               "funder_causes", "funder_areas", "funder_beneficiaries", "funder_grants", 
               "financials", "funder_financials"]

for table in tables:
    globals()[table] = get_table_from_supabase(url, key, table)

#get recipients with filter
recipients = get_table_from_supabase(url, key, "recipients", batch_size=50, filter_recipients=True)
all_recipient_ids = set(recipients["recipient_id"].unique())

#get and filter recipient join tables
recipient_join_tables = ["recipient_grants", "recipient_areas", "recipient_beneficiaries", "recipient_causes"]
for table in recipient_join_tables:
    df = get_table_from_supabase(url, key, table)
    globals()[table] = df[df["recipient_id"].isin(all_recipient_ids)]

print(f"Loaded {len(funders)} funders, {len(recipients)} recipients, {len(grants)} grants")
print(f"Filtered recipient join tables to {len(all_recipient_ids)} valid recipients")

## The Funders Dataframe

### Main Table

In [ ]:
funders_df = funders.copy()

#define table relationships for funders
funder_rels = [
    {
        "join_table": funder_causes,
        "lookup_table": causes,
        "key": "cause_id",
        "value_col": "cause_name",
        "result_col": "causes"
    },
    {
        "join_table": funder_areas,
        "lookup_table": areas,
        "key": "area_id",
        "value_col": "area_name",
        "result_col": "areas"
    },
    {
        "join_table": funder_beneficiaries,
        "lookup_table": beneficiaries,
        "key": "ben_id",
        "value_col": "ben_name",
        "result_col": "beneficiaries"
    }
]

#group and merge
for rel in funder_rels:
    grouped = rel["join_table"].merge(rel["lookup_table"], on=rel["key"])
    grouped = grouped.groupby("registered_num")[rel["value_col"]].apply(list).reset_index()
    grouped.columns = ["registered_num", rel["result_col"]]
    funders_df = funders_df.merge(grouped, on="registered_num", how="left")

#add grant statistics columns 
grants_stats = funder_grants.merge(grants, on="grant_id")
grants_agg = grants_stats.groupby("registered_num").agg({
    "grant_id": "count",
    "amount": ["sum", "mean", "median"]
}).reset_index()
grants_agg.columns = ["registered_num", "num_grants", "total_given", "avg_grant", "median_grant"]

funders_df = funders_df.merge(grants_agg, on="registered_num", how="left")
funders_df["num_grants"] = funders_df["num_grants"].astype("Int64")

#replace nan values with empty lists
funders_df["causes"] = funders_df["causes"].apply(lambda x: x if isinstance(x, list) else [])
funders_df["areas"] = funders_df["areas"].apply(lambda x: x if isinstance(x, list) else [])
funders_df["beneficiaries"] = funders_df["beneficiaries"].apply(lambda x: x if isinstance(x, list) else [])

#round to 2 decimal places
funders_df = funders_df.round(2)
pd.set_option("display.float_format", "{:.2f}".format)

#format financial columns
float_cols = ["income_latest", "expenditure_latest", "total_given", "avg_grant", "median_grant"]
for col in float_cols:
    if col in funders_df.columns:
        funders_df[col + "_gbp"] = funders_df[col].apply(add_gbp_columns)

### Financial History Table

In [ ]:
#get full financial records and separate into income and expenditure
financial_history = funder_financials.merge(financials, on="financials_id")
income_history = financial_history[financial_history["financials_type"] == "income"]
expenditure_history = financial_history[financial_history["financials_type"] == "expenditure"]

#make financials dicts
income_by_funder = income_history.groupby("registered_num").apply(
    lambda x: dict(zip(x["financials_year"], x["financials_value"]))
).reset_index()
income_by_funder.columns = ["registered_num", "income_history"]

expenditure_by_funder = expenditure_history.groupby("registered_num").apply(
    lambda x: dict(zip(x["financials_year"], x["financials_value"]))
).reset_index()
expenditure_by_funder.columns = ["registered_num", "expenditure_history"]

#merge with funders and replace nans
funders_df = funders_df.merge(income_by_funder, on="registered_num", how="left")
funders_df = funders_df.merge(expenditure_by_funder, on="registered_num", how="left")
funders_df["income_history"] = funders_df["income_history"].apply(lambda x: x if isinstance(x, dict) else {})
funders_df["expenditure_history"] = funders_df["expenditure_history"].apply(lambda x: x if isinstance(x, dict) else {})

### The List Entries

In [ ]:
#get list entries
list_entries = get_table_from_supabase(url, key, "list_entries")
funder_list = get_table_from_supabase(url, key, "funder_list")
list_with_info = funder_list.merge(list_entries, on="list_id")

#get list of entries for each funder
list_grouped = list_with_info.groupby("registered_num")["list_info"].apply(list).reset_index()
list_grouped.columns = ["registered_num", "list_entries"]

#merge with funders and replace nans
funders_df = funders_df.merge(list_grouped, on="registered_num", how="left")
funders_df["list_entries"] = funders_df["list_entries"].apply(lambda x: x if isinstance(x, list) else [])

In [ ]:
#extend column view, sort and preview funders
pd.set_option("display.max_columns", 100)
funders_df = funders_df.sort_values("total_given_gbp", ascending=False)
funders_df.head()

## The Grants Dataframe

### Main Table

In [ ]:
grants_df = grants.copy()

#add funder info
grants_df = grants_df.merge(funder_grants, on="grant_id")
grants_df = grants_df.merge(funders[["registered_num", "name"]], on="registered_num")
grants_df = grants_df.rename(columns={"name": "funder_name"})
grants_df = grants_df.rename(columns={"registered_num": "funder_num"})

#add recipient info  
grants_df = grants_df.merge(recipient_grants, on="grant_id")
grants_df = grants_df.merge(recipients[["recipient_id", "recipient_name", "recipient_activities"]], 
                        on="recipient_id", 
                        how="left")

#add recipient areas
recip_areas_grouped = recipient_areas.merge(areas, on="area_id")
recip_areas_grouped = recip_areas_grouped.groupby("recipient_id")["area_name"].apply(list).reset_index()
recip_areas_grouped.columns = ["recipient_id", "recipient_areas"]
grants_df = grants_df.merge(recip_areas_grouped, on="recipient_id", how="left")

#add recipient causes
recip_causes_grouped = recipient_causes.merge(causes, on="cause_id")
recip_causes_grouped = recip_causes_grouped.groupby("recipient_id")["cause_name"].apply(list).reset_index()
recip_causes_grouped.columns = ["recipient_id", "recipient_causes"]
grants_df = grants_df.merge(recip_causes_grouped, on="recipient_id", how="left")

#add recipient beneficiaries
recip_beneficiaries_grouped = recipient_beneficiaries.merge(beneficiaries, on="ben_id")
recip_beneficiaries_grouped = recip_beneficiaries_grouped.groupby("recipient_id")["ben_name"].apply(list).reset_index()
recip_beneficiaries_grouped.columns = ["recipient_id", "recipient_beneficiaries"]
grants_df = grants_df.merge(recip_beneficiaries_grouped, on="recipient_id", how="left")

#replace nan values with empty lists
if "recipient_areas" in grants_df.columns:
    grants_df["recipient_areas"] = grants_df["recipient_areas"].apply(lambda x: x if isinstance(x, list) else [])
if "recipient_causes" in grants_df.columns:
    grants_df["recipient_causes"] = grants_df["recipient_causes"].apply(lambda x: x if isinstance(x, list) else [])
if "recipient_beneficiaries" in grants_df.columns:
    grants_df["recipient_beneficiaries"] = grants_df["recipient_beneficiaries"].apply(lambda x: x if isinstance(x, list) else [])

#add source of grant
grants_df["source"] = grants_df["grant_id"].apply(lambda x: "Accounts" if str(x).startswith("2") else "360Giving")

#round to 2 decimal places
grants_df = grants_df.round(2)

#format financial columns
grants_df["amount_gbp"] = grants_df["amount"].apply(add_gbp_columns)

In [ ]:
#sort and preview grants
grants_df = grants_df.sort_values("grant_title", ascending=True)
grants_df.head()

-------

# Descriptive Statistics

## Summary Statistics

In [ ]:
#build df of summary statistics
summary_data = make_summary_df(funders_df, grants_df)
summary_df = pd.DataFrame(summary_data)
summary_df["Value"] = summary_df.apply(format_stats, axis=1)
summary_df = format_df(summary_df)

## Calculated Statistics

In [ ]:
#get calculated stats and build df
stats = calculate_stats(funders_df, grants_df)
calculated_data = make_calculated_df(stats)
calculated_df = pd.DataFrame(calculated_data)
calculated_df["Value"] = calculated_df.apply(format_stats, axis=1)
calculated_df = format_df(calculated_df)

-----

# Data Quality

## 1. Missingness

I will view the structure of each dataframe to check for missing data and confirm that datatypes are correct.

### Analysis of Missingness in Funders Dataframe

In [ ]:
funders_df.info()

In [ ]:
#missing activities - check manually
missing_activities = funders_df[funders_df["activities"].isna()]["registered_num"].tolist()
print(missing_activities)

In [ ]:
#missing sections - check proportion of funders with sections extracted from accounts
has_sections = funders_df[
    funders_df["objectives_activities"].notna() |
    funders_df["achievements_performance"].notna() |
    funders_df["grant_policy"].notna()
]

has_sections_total = len(has_sections)
has_sections_proportion = has_sections_total / 996

print(f"Funders with accessible accounts: 327")
print(f"Funders with extracted sections: {has_sections_total}")
print(f"Proportion of funders with sections: {has_sections_proportion:.2%}")


### Exploration of Findings from Missingness Analysis (Funders)

There are five funders in the database with empty `activities`. I have checked the Charity Commission website and it does appear that these funders have simply not declared any activities. They are all relatively new having submitted only one set of accounts. The `activities_objectives` column has not been populated for any of these funders, which is unfortunate as this could have provided a further source of information. One of these funders has a website so, if I am able to achieve my stretch target of scraping websites, this may offer details (although at the time of writing, the website does not exist).

It is expected that a significant proportion of funders would not have a website, and no action is required to address this.

**Problem with Accounts**

My database building scripts (for PDFs) have a serious limitation as I am unable to scrape the Charity Commission website for accounts where the page contains JavaScript. The older pages, which are basic HTML, are accessible but the newer ones are not, and I am unable to tell which charities have been updated to the new system until the script attempts to scrape them and fails. Having built the database, I can now see from the print statements that 327 funders have accessible accounts, representing 32.8% of the 996 funders in the sample. My calculations above show that the required sections have been extracted from accounts for 24.4% of funders - varying from 52 funders with `grant_policy`, to 194 funders with `achievements_peformance` and 230 funders with `objectives_activities`.

### Analysis of Missingness in Grants Dataframe

In [ ]:
grants_df.info()

In [ ]:
#missing grant title and description - check numbers per source
missing_by_source = grants_df.groupby("source").agg({
    "grant_title": lambda x: x.isna().sum(),
    "grant_desc": lambda x: x.isna().sum(),
    "grant_id": "count"
}).rename(columns={"grant_id": "total_grants", "grant_title": "missing_title", "grant_desc": "missing_desc"})

print(missing_by_source)

In [ ]:
#missing grant amounts - spotcheck manually
missing_amounts = grants_df[grants_df["amount"].isna()]["funder_num"].tolist()
print(missing_amounts)

In [ ]:
#missing recipient activities - compare by source
missing_recips = grants_df["recipient_activities"].isna().sum()
accounts_recips = grants_df[
    (grants_df["source"] == "Accounts") &
    (grants_df["recipient_activities"].isna())
]
missing_360g = grants_df[
    (grants_df["source"] == "360Giving") &
    (grants_df["recipient_activities"].isna())
]

print(f"Grants extracted from accounts: {len(accounts_recips):,} ({len(accounts_recips)/missing_recips*100:.1f}%)")
print(f"360Giving recipients with missing activities: {len(missing_360g):,} ({len(missing_360g)/missing_recips*100:.1f}%)")

### Exploration of Findings from Missingness Analysis (Grants)

There are 33,147 rows in the dataframe, 287 more than the number of unique grants as some grants are given to multiple recipients. The deduplication script in `stats_builder.py` ensures that the value of each grant has only been counted once for the purpose of the summary/calculated statistics.

I have explored the missing data from `grant_title` and `grant_desc` to check that only grants extracted from accounts are missing these fields. This was to be expected as these details are not available for the grants from accounts; the figures above confirm that every grant from the 360Giving API is complete with a title and description. 

Of the 32,815 grants in the database, 46 (0.14%) are missing `amount`. I have performed a spot-check of 12 grants (25%), which suggests that these are genuine extraction errors from funder accounts rather than systematic issues. I have decided to keep these 46 grants in the database as they still provide valuable information about funder-recipient relationships and giving patterns, and such a small number of missing amounts is not likely to affect analyses/models.

I also explored the 17,318 rows in `grants_df` with missing `recipient_activities`. Null values in this column are to be expected for recipients that are added following extraction from PDFs, which accounts for 11,486 (66.3%). The remaining 5,832 (33.7%) null values - which are present within records added to the database by the 360Giving API - can be explained by:
- Charities that have been removed from the Charity Commission, e.g. if they have closed down
- Registered charities that simply do not declare their activities (as present in 5 records in `funders_df`)
- Where abnormalities are evident in `registered_num`. For example, the RSPB is identified in the 360Giving API by *207076 & SC037654* - its registered numbers for both the Charity Commission of England & Wales and the Office of the Scottish Charity Regulator. My database builder script was unable to match on *207076 & SC037654* as `registered_num`. 

-------

## 2. Word Counts

I will check the lengths of the shortest and longest text entries, to ensure that they have been imported correctly and are not too short or long for unexpected reasons. Many funders provide very short explanations of their activities/objectives etc., such as simply "grant-giving" which is just one word - so this would not be abnormal. I will confirm that particularly long entries are not corrupted or the result of multiple documents being combined accidentally.

### Analysis of Word Counts in Funders Dataframe

In [ ]:
#check word counts for text columns in funders df
funders_text_cols = ["activities", "objectives", "objectives_activities", "achievements_performance", "grant_policy"]

#create columns
for col in funders_text_cols:
    funders_df[f"word_count_{col}"] = funders_df[col].str.split().str.len()
for col in funders_text_cols:
    print(f"{col.upper()}")
    print(f"{'_'*30}\n")

    word_count_col = f"word_count_{col}"
    not_nas = funders_df[funders_df[word_count_col].notna()]

    #get minimums and maximums for each text column
    if len(not_nas) > 0:
        min_idx = not_nas[word_count_col].idxmin()
        max_idx = not_nas[word_count_col].idxmax()

        examples = funders_df.loc[[min_idx, max_idx], ["registered_num", "name", word_count_col, col]]
        examples.index = ["Minimum", "Maximum"]

        display(examples)
    else:
        print("No data available\n")

#### Word Counts for Sections Returned from Charity Commission API

In [ ]:
#high word counts - check top 10 longest sections from API (to manually check)
long_activities = get_longest_values(funders_df, "word_count_activities", "registered_num")
long_objectives = get_longest_values(funders_df, "word_count_objectives", "registered_num")

api_sections_long = [
    ("activities", long_activities),
    ("objectives", long_objectives)
]

for name, longest in api_sections_long:
    print(f"Top 10 longest {name}: {longest}")

In [ ]:
#low word counts - count and view short sections from API
api_sections_short = {
    "activities": "Short activities sections",
    "objectives": "Short objectives sections"
}

for col, title in api_sections_short.items():
    funders_df[f"short_{col}_section"] = (
        (funders_df[col].str.split().str.len() < 5) &
        (funders_df[col].str.upper().str.strip() != "GENERAL CHARITABLE PURPOSES")
    )
    short_sections = funders_df[funders_df[f"short_{col}_section"]][col].dropna().unique()
    
    print(f"\n{title}: {funders_df[f'short_{col}_section'].sum():,}")
    print_in_rows(short_sections, 5)

In [ ]:
#low word counts - check activities containing "NONE"
activities_none = funders_df["activities"].str.contains("NONE", na=False)
funders_df[activities_none]

#### Word Counts for Sections Extracted from PDF Accounts

In [ ]:
#high word counts - check top 10 longest extracted sections (to manually check)
long_obj_act = get_longest_values(funders_df, "word_count_objectives_activities", "registered_num")
long_ach_perf = get_longest_values(funders_df, "word_count_achievements_performance", "registered_num")
long_grant_policy = get_longest_values(funders_df, "word_count_grant_policy", "registered_num")

accounts_sections_long = [
    ("objectives_activities", long_obj_act),
    ("achievements_performance", long_ach_perf),
    ("grant_policy", long_grant_policy)
]

for name, longest in accounts_sections_long:
    print(f"Top 10 longest {name}: {longest}")

In [ ]:
#low word counts - count and view short sections from accounts
accounts_sections_short = {
    "objectives_activities": "Short objectives_activities sections",
    "achievements_performance": "Short achievements_performance sections",
    "grant_policy": "Short grant_policy sections"
}

for col, title in accounts_sections_short.items():
    funders_df[f"short_{col}_section"] = (
        (funders_df[col].str.split().str.len() < 20)
    )
    short_sections = funders_df[funders_df[f"short_{col}_section"]][col].dropna().unique()
    
    print(f"\n{title}: {funders_df[f'short_{col}_section'].sum():,}")
    print_in_rows(short_sections, 2)

### Exploration of Findings from Word Count Analysis (Funders)

**Sections from API**

Having manually checked the maximum values against their Charity Commission records, I am confident that there are no entries that are concerningly long. The top 10 longest `activities` are not abnormal; they are just relatively wordy compared to others. I viewed the shortest values (fewer than five words) and concluded that they are acceptable, although I will reassign the values containing "NONE" to N/A to improve the quality of the embedding.

**Sections from Accounts**

The word count analysis has highlighted that there are numerous issues with the sections that have been extracted from accounts. Many that are too long contain multiple sections, whereas many that are too short have been cut off prematurely. There is also a problem of repetition - the extractor sometimes extracts the same (excessively long) chunk of text for both `objectives_activities` and `achievements_performance`, defeating the purpose. Looking at the top 10 shortest and longest has demonstrated that the regex extraction technique employed in the database builder has not worked very well. Rather than rebuilding the entire PDF extraction pipeline, I will address this data quality issue by re-processing the extracted text through an LLM-based extractor with the aim of better handling the variation in section sizes and formatting. The LLM should be better at the fuzzy matching that regex has struggled with.

### Analysis of Word Counts in Grants Dataframe

In [ ]:
#check word counts for text columns in grants df
grants_text_cols = ["grant_title", "grant_desc", "recipient_activities"]

#create columns
for col in grants_text_cols:
    grants_df[f"word_count_{col}"] = grants_df[col].str.split().str.len()
for col in grants_text_cols:
    print(f"{col.upper()}")
    print(f"{'_'*30}\n")

    word_count_col = f"word_count_{col}"
    not_nas = grants_df[grants_df[word_count_col].notna()]

    #get minimums and maximums for each text column
    if len(not_nas) > 0:
        min_idx = not_nas[word_count_col].idxmin()
        max_idx = not_nas[word_count_col].idxmax()

        examples = grants_df.loc[[min_idx, max_idx], ["funder_num", "funder_name", "grant_id", "recipient_name", word_count_col, col]]
        examples.index = ["Minimum", "Maximum"]

        display(examples)
    else:
        print("No data available\n")

In [ ]:
#low word counts - count and view short grant details from 360Giving API/CC public extract
grant_details = {
    "grant_title": "Short grant title",
    "grant_desc": "Short grant description",
    "recipient_activities": "Short recipient activities"
}

for col, title in grant_details.items():
    grants_df[f"short_{col}_section"] = (
        (grants_df[col].str.split().str.len() < 3)
    )
    short_grant_details = grants_df[grants_df[f"short_{col}_section"]][col].dropna().unique()
    
    # print(f"\n{title}: {grants_df[f'short_{col}_section'].sum():,}")
    # print_in_rows(short_grant_details, 12)

In [ ]:
#high word counts - check top 10 longest grant details (to manually check)
long_grant_title = get_longest_values(grants_df, "word_count_grant_title", "grant_id")
long_grant_desc = get_longest_values(grants_df, "word_count_grant_desc", "grant_id")
long_recip_activities = get_longest_values(grants_df, "word_count_recipient_activities", "grant_id")

sections = [
    ("grant_title", long_grant_title),
    ("grant_desc", long_grant_desc),
    ("recipient_activities", long_recip_activities)
]

for name, longest in sections:
    print(f"Top 10 longest {name}: {longest}")

### Exploration of Findings from Word Count Analysis (Grants)

The word count analysis on the grants dataframe shows that there are almost 2,000 entries with a length of fewer than three words. Having inspected these, I can see that there are a small number of unusual values that need to be cleaned, such as an entry in `recipient_activities` that reads *#NAME?*. There are also several hundred grant titles that appear to be IDs as opposed to descriptive titles (e.g. *VFTF R6-SOLACE-2024*); these will not lend themselves to embedding, so I will remove them. They all appear to end in a dash and a year, so can be identified by "-2". 

In terms of high word counts, there are no concerns. As with the information called from the API in the funders dataframe, the longest grant details are simply relatively long and wordy. I am confident that the information from the 360Giving API is highly reliable, with a very small minority of unexpected values that can be cleaned easily.

---------

## 3. Quality of Data from APIs

Although the data that has been returned from the APIs is generally clean and reliable, there are some minor issues that require tidying.

### Problems Identified by Word Count Analyses

In [ ]:
#replace ID grant titles with None
grants_df.loc[grants_df["grant_title"].str.contains("-2", na=False), "grant_title"] = None

In [ ]:
#replace unusual values with None
unusual_values = ["#NAME?", "NIL ACTIVITY.", ".", "AS BEFORE", "IN LIQUIDATION", "NONE", "0", "N/A"]
grants_df["grant_desc"] = grants_df["grant_desc"].replace(unusual_values, None)
grants_df["recipient_activities"] = grants_df["recipient_activities"].replace(unusual_values, None)

In [ ]:
#print grant details again to confirm cleaning has been successful
grant_details = {
    "grant_title": "Short grant title",
    "grant_desc": "Short grant description",
    "recipient_activities": "Short recipient activities"
}

for col, title in grant_details.items():
    grants_df[f"short_{col}_section"] = (
        (grants_df[col].str.split().str.len() < 3)
    )
    short_grant_details = grants_df[grants_df[f"short_{col}_section"]][col].dropna().unique()
    
    # print(f"\n{title}: {grants_df[f'short_{col}_section'].sum():,}")
    # print_in_rows(short_grant_details, 12)

### Checking and Cleaning Other Variables (Funders)

In [ ]:
#check funder names
name_lengths = funders_df["name"].str.len()

print(f"Funders with names < 10 characters: {(name_lengths < 10).sum()}")
print(f"Funders with names > 75 characters: {(name_lengths > 75).sum()}")

print("\nTop 10 shortest funder names (< 10 characters)")
short_names = funders_df[name_lengths < 10][["registered_num", "name", "website"]].sort_values("name", key=lambda x: x.str.len()).head(10)
display(short_names)

print("\nTop 10 longest funder names (> 75 characters)")
long_names = funders_df[name_lengths > 100][["registered_num", "name", "website"]].sort_values("name", key=lambda x: x.str.len(), ascending=False)
display(long_names)


In [ ]:
#check funder websites
website_lengths = funders_df["website"].str.len()

print(f"Funders with websites < 15 characters: {(website_lengths < 15).sum()}")
print(f"Funders with websites > 75 characters: {(website_lengths > 75).sum()}")

print("\nTop 10 shortest funder websites (< 15 characters)")
short_websites = funders_df[website_lengths < 15][["registered_num", "name", "website"]].sort_values("website", key=lambda x: x.str.len()).head(10)
display(short_websites)

print("\nTop 10 longest funder websites (> 75 characters)")
long_websites = funders_df[website_lengths > 75][["registered_num", "name", "website"]].sort_values("website", key=lambda x: x.str.len(), ascending=False)
display(long_websites)

In [ ]:
#reassign null to incorrectly stored urls
funders_df.loc[website_lengths < 15, "website"] = None

### Checking and Cleaning Other Variables (Grants)

In [ ]:
#check recipient ids - view ids with unexpected starting characters
invalid_ids = grants_df[~grants_df["recipient_id"].str.startswith(("1", "2", "3", "4", "5", "6", "7", "8", "9", "P"), na=False)]["recipient_id"].unique()
# print(list(invalid_ids))

In [ ]:
#remove incorrect spaces and 0s
grants_df["recipient_id"] = grants_df["recipient_id"].astype(str).str.strip()
grants_df["recipient_id"] = grants_df["recipient_id"].str.lstrip("0")

#reassign remaining invalid ids
invalid_ids = ~grants_df["recipient_id"].str.startswith(("1", "2", "3", "4", "5", "6", "7", "8", "9", "P"), na=False)
grants_df.loc[invalid_ids, "recipient_id"] = [f"invalid_{i}" for i in range(invalid_ids.sum())]
invalid_ids = grants_df[~grants_df["recipient_id"].str.startswith(("1", "2", "3", "4", "5", "6", "7", "8", "9", "P", "i"), na=False)]["recipient_id"].unique()
print(list(invalid_ids))

### Explanation of Cleaning Process

**Funder Names and Websites**

I am satisfied that there are no concerns with the length of funder names. Looking at the shortest websites, there appeared to be one invalid URL, and five that are stored in an incorrect null format; I reassigned all six of these to None.

**Recipient IDs**

Some values in `recipient_id` need to be cleaned. There is a small subset with incorrect starting characters, and several that are not registered charity numbers. Working with organisations that are not registered with the Charity Commission (of England & Wales) is out of the scope of this project. Where there is clearly an error in the Charity Commission registered number, I have cleaned these values. I have removed spaces and 0s from the start of IDs and reassigned any other values (e.g. *N/A* or *Exempt*) to show that the ID was invalid. Where `recipient_id` is a company number or a registered number from Scotland or Northern Ireland, I have also assigned an "invalid" ID. The text content of these records can be embedded as-is, but anything else (e.g. calling a different API) is out of the scope of the project so no additional value will be gleaned from keeping the original IDs.

## 4. Quality of Extracted Sections and Grants

To extract sections (i.e `activities_objectives`, `achievements_performance` and `grants_policy`) and grants, I have relied on regex matching and the Claude API (using the Haiku 3 model). Unfortunately, there are noteable errors and quality issues that would degrade the quality of embeddings and undermind the reliability of semantic matching. As such, they cannot be ignored. 

In [ ]:
#load reprocessed sections
sections_df = pd.read_csv("llm_sections.csv")
sections_df.head()

In [ ]:
#check efficacy of sections reprocessor
sections_df.info()

### Recipient Names

The `recipient_name` variable has been built from two sources. The first is the [Charity Commission's Public Extract](https://register-of-charities.charitycommission.gov.uk/en/register/full-register-download), which is updated daily and includes registered charities' names as logged with the Charity Commission. These names are generally clean and standardised, though charities occasionally use trading names that differ from their official registered names.

The second source is the organisation's name as extracted from funders' accounts by the Claude API. This is significantly less reliable and is a noted limitation of this project. The reliability is affected in part by the reliance on a large language model and its fuzzy intepretations of messy text, but also due to the fact that funders' accounts are not held to any standard of enforcement in terms of the correct spelling, uniformity, or indeed accuracy of recipients' names. Examples that have been noted during the course of this project include inconsistency in pluralisation and spacing (e.g. *hospices* vs *hospice*, *Tearfund* vs *Tear Fund*); missing words (e.g. *British Red Cross Society* vs *British Red Cross*); errors in punctuation (e.g. *soldiers'* vs *solider's*); and inconsistency in word order (e.g. *Lincoln University* vs *University of Lincoln*).

The LLM has also at times failed to interpret account entries properly, extracting purchase descriptions as recipient names (e.g. 'cricket balls for the Year 7 team' from a PTA's accounts) or recording vague summaries as distinct recipients (e.g. 'five various causes'). These errors are particularly prevalent in accounts from single-beneficiary funders where expenditure descriptions differ structurally from multi-recipient grant listings.

To do:
- Recipients with weird names
- Recipients who are individuals - data protection issue
- Unusual characters in text that will be embedded

- Single beneficiary funders

-------

# Exploration of Giving Patterns and Funder Characteristics

## 1. Categories

One of the key objectives of prospie is to help fundraisers navigate the confusing trusts landscape, which is confusing largely due to the mismatch of information that is available. Funders may indicate a particular cause or area of activity, but these do not always align with their actual giving habits. I will therefore compare the classifications that funders state (their identified causes, beneficiaries, and areas of activity) with those of the recipients who are awarded their grants.

In [ ]:
#display plots to compare funders' and recipients' classifications
fig, axes = plt.subplots(3, 2, figsize=(18, 15))

#causes
make_bar_chart(grants_df, "recipient_causes", "Recipients: Most Popular Causes", color="#2E86AB", ax=axes[0, 0])
make_bar_chart(funders_df, "causes", "Funders: Most Popular Causes", color="#2E86AB", ax=axes[0, 1])

#areas
make_bar_chart(grants_df, "recipient_areas", "Recipients: Most Popular Areas", color="#A23B72", ax=axes[1, 0])
make_bar_chart(funders_df, "areas", "Funders: Most Popular Areas", color="#A23B72", ax=axes[1, 1])

#beneficiaries
make_bar_chart(grants_df, "recipient_beneficiaries", "Recipients: Most Popular Beneficiaries", color="#F18F01", ax=axes[2, 0])
make_bar_chart(funders_df, "beneficiaries", "Funders: Most Popular Beneficiaries", color="#F18F01", ax=axes[2, 1])

plt.tight_layout()
plt.subplots_adjust(hspace=0.4)
plt.show()

It can be observed that funders often state an interest in general causes, areas and beneficiaries - many do not specify particular interests and state that they will consider applications from any area of the sector. Further analysis will be useful, particularly following the creation of embeddings, to understand the practical reality of their funding priorities, which may reveal implicit preferences or local biases not reflected in their published criteria.

### Relationships between Categories

## Temporal and Financial Analysis